In [1]:
import math
import numpy as np
import pandas as pd
import scipy.stats as scs
import statsmodels.api as sm
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rcParams['font.family'] = 'serif'

In [2]:
# Analyzing Returns from Geometric Brownian Motion

In [3]:
#
# Analyzing Returns from Geometric Brownian Motion
# 03_stf/GBM_returns.py
#
# (c) Dr. Yves J. Hilpisch
# Derivatives Analytics with Python
#


#
# Helper Function
#


def dN(x, mu, sigma):
    ''' Probability density function of a normal random variable x.

    Parameters
    ==========
    mu : float
        expected value
    sigma : float
        standard deviation

    Returns
    =======
    pdf : float
        value of probability density function
    '''
    z = (x - mu) / sigma
    pdf = np.exp(-0.5 * z ** 2) / math.sqrt(2 * math.pi * sigma ** 2)
    return pdf


#
# Simulate a Number of Years of Daily Stock Quotes
#


def simulate_gbm():
    # model parameters
    S0 = 100.0  # initial index level
    T = 10.0  # time horizon
    r = 0.05  # risk-less short rate
    vol = 0.2  # instantaneous volatility

    # simulation parameters
    np.random.seed(250000)
    gbm_dates = pd.DatetimeIndex(start='30-09-2004',
                                 end='30-09-2014',
                                 freq='B')
    M = len(gbm_dates)  # time steps
    I = 1  # index level paths
    dt = 1 / 252.  # fixed for simplicity
    df = math.exp(-r * dt)  # discount factor

    # stock price paths
    rand = np.random.standard_normal((M, I))  # random numbers
    S = np.zeros_like(rand)  # stock matrix
    S[0] = S0  # initial values
    for t in range(1, M):  # stock price paths
        S[t] = S[t - 1] * np.exp((r - vol ** 2 / 2) * dt +
                                 vol * rand[t] * math.sqrt(dt))

    gbm = pd.DataFrame(S[:, 0], index=gbm_dates, columns=['index'])
    gbm['returns'] = np.log(gbm['index'] / gbm['index'].shift(1))

    # Realized Volatility (eg. as defined for variance swaps)
    gbm['rea_var'] = 252 * np.cumsum(gbm['returns'] ** 2) / np.arange(len(gbm))
    gbm['rea_vol'] = np.sqrt(gbm['rea_var'])
    gbm = gbm.dropna()
    return gbm

# Return Sample Statistics and Normality Tests


def print_statistics(data):
    print("RETURN SAMPLE STATISTICS")
    print("---------------------------------------------")
    print("Mean of Daily  Log Returns %9.6f" % np.mean(data['returns']))
    print("Mean of Annua. Log Returns %9.6f" %
          (np.mean(data['returns']) * 252))
    print("Std  of Annua. Log Returns %9.6f" %
          (np.std(data['returns']) * math.sqrt(252)))
    print("---------------------------------------------")
    print("Skew of Sample Log Returns %9.6f" % scs.skew(data['returns']))
    print("Skew Normal Test p-value   %9.6f" %
          scs.skewtest(data['returns'])[1])
    print("---------------------------------------------")
    print("Kurt of Sample Log Returns %9.6f" % scs.kurtosis(data['returns']))
    print("Kurt Normal Test p-value   %9.6f" %
          scs.kurtosistest(data['returns'])[1])
    print("---------------------------------------------")
    print("Normal Test p-value        %9.6f" %
          scs.normaltest(data['returns'])[1])
    print("---------------------------------------------")
    print("Realized Volatility        %9.6f" % data['rea_vol'].iloc[-1])
    print("Realized Variance          %9.6f" % data['rea_var'].iloc[-1])

#
# Graphical Output
#

# daily quotes and log returns


def quotes_returns(data):
    ''' Plots quotes and returns. '''
    plt.figure(figsize=(9, 6))
    plt.subplot(211)
    data['index'].plot()
    plt.ylabel('daily quotes')
    plt.grid(True)
    plt.axis('tight')

    plt.subplot(212)
    data['returns'].plot()
    plt.ylabel('daily log returns')
    plt.grid(True)
    plt.axis('tight')

# histogram of annualized daily log returns


def return_histogram(data):
    ''' Plots a histogram of the returns. '''
    plt.figure(figsize=(9, 5))
    x = np.linspace(min(data['returns']), max(data['returns']), 100)
    plt.hist(np.array(data['returns']), bins=50, normed=True)
    y = dN(x, np.mean(data['returns']), np.std(data['returns']))
    plt.plot(x, y, linewidth=2)
    plt.xlabel('log returns')
    plt.ylabel('frequency/probability')
    plt.grid(True)

# Q-Q plot of annualized daily log returns


def return_qqplot(data):
    ''' Generates a Q-Q plot of the returns.'''
    plt.figure(figsize=(9, 5))
    sm.qqplot(data['returns'], line='s')
    plt.grid(True)
    plt.xlabel('theoretical quantiles')
    plt.ylabel('sample quantiles')


# realized volatility
def realized_volatility(data):
    ''' Plots the realized volatility. '''
    plt.figure(figsize=(9, 5))
    data['rea_vol'].plot()
    plt.ylabel('realized volatility')
    plt.grid(True)

# mean return, volatility and correlation (252 days moving = 1 year)


def rolling_statistics(data):
    ''' Calculates and plots rolling statistics (mean, std, correlation). '''
    plt.figure(figsize=(11, 8))

    plt.subplot(311)
    mr = data['returns'].rolling(252).mean() * 252
    mr.plot()
    plt.grid(True)
    plt.ylabel('returns (252d)')
    plt.axhline(mr.mean(), color='r', ls='dashed', lw=1.5)

    plt.subplot(312)
    vo = data['returns'].rolling(252).std() * math.sqrt(252)
    vo.plot()
    plt.grid(True)
    plt.ylabel('volatility (252d)')
    plt.axhline(vo.mean(), color='r', ls='dashed', lw=1.5)
    vx = plt.axis()

    plt.subplot(313)
    co = mr.rolling(252).corr(vo)
    co.plot()
    plt.grid(True)
    plt.ylabel('correlation (252d)')
    cx = plt.axis()
    plt.axis([vx[0], vx[1], cx[2], cx[3]])
    plt.axhline(co.mean(), color='r', ls='dashed', lw=1.5)


In [4]:
# Read Data for DAX from the Web


def read_dax_data():
    ''' Reads historical DAX data from Yahoo! Finance, calculates log returns,
    realized variance and volatility.'''
    DAX = pd.read_csv('http://hilpisch.com/tr_eikon_eod_data_long.csv',
                      index_col=0, parse_dates=True)['.GDAXI']
    DAX = pd.DataFrame(DAX)
    DAX = DAX.loc['2004-09-30':'2014-09-30']
    DAX.rename(columns={'.GDAXI': 'index'}, inplace=True)
    DAX['returns'] = np.log(DAX['index'] / DAX['index'].shift(1))
    DAX['rea_var'] = 252 * np.cumsum(DAX['returns'] ** 2) / np.arange(len(DAX))
    DAX['rea_vol'] = np.sqrt(DAX['rea_var'])
    DAX = DAX.dropna()
    return DAX


def count_jumps(data, value):
    ''' Counts the number of return jumps as defined in size by value. '''
    jumps = np.sum(np.abs(data['returns']) > value)
    return jumps


In [5]:
read_dax_data()

,index,returns,rea_var,rea_vol
Date,,,,
2004-10-01,3994.96,0.025879,0.168773,0.410819
2004-10-04,4033.28,0.009546,0.095869,0.309627
2004-10-05,4048.71,0.003818,0.065137,0.255220
2004-10-06,4049.66,0.000235,0.048857,0.221035
2004-10-07,4043.36,-0.001557,0.039207,0.198009
...,...,...,...,...
2014-09-24,9661.97,0.006952,0.046055,0.214604
2014-09-25,9510.01,-0.015853,0.046061,0.214619
2014-09-26,9490.55,-0.002048,0.046044,0.214579


In [6]:
#
# Valuation of European Call Options in BSM Model
# and Numerical Derivation of Implied Volatility
# 03_stf/BSM_imp_vol.py
#
# (c) Dr. Yves J. Hilpisch
# from Hilpisch, Yves (2014): Python for Finance, O'Reilly.
#
from math import log, sqrt, exp
from scipy import stats
from scipy.optimize import fsolve


class call_option(object):
    ''' Class for European call options in BSM Model.

    Attributes
    ==========
    S0 : float
        initial stock/index level
    K : float
        strike price
    t : datetime/Timestamp object
        pricing date
    M : datetime/Timestamp object
        maturity date
    r : float
        constant risk-free short rate
    sigma : float
        volatility factor in diffusion term

    Methods
    =======
    value : float
        return present value of call option
    vega : float
        return vega of call option
    imp_vol : float
        return implied volatility given option quote
    '''

    def __init__(self, S0, K, t, M, r, sigma):
        self.S0 = float(S0)
        self.K = K
        self.t = t
        self.M = M
        self.r = r
        self.sigma = sigma

    def update_ttm(self):
        ''' Updates time-to-maturity self.T. '''
        if self.t > self.M:
            raise ValueError("Pricing date later than maturity.")
        self.T = (self.M - self.t).days / 365.

    def d1(self):
        ''' Helper function. '''
        d1 = ((log(self.S0 / self.K)
               + (self.r + 0.5 * self.sigma ** 2) * self.T)
              / (self.sigma * sqrt(self.T)))
        return d1

    def value(self):
        ''' Return option value. '''
        self.update_ttm()
        d1 = self.d1()
        d2 = ((log(self.S0 / self.K)
               + (self.r - 0.5 * self.sigma ** 2) * self.T)
              / (self.sigma * sqrt(self.T)))
        value = (self.S0 * stats.norm.cdf(d1, 0.0, 1.0)
                 - self.K * exp(-self.r * self.T) * stats.norm.cdf(d2, 0.0, 1.0))
        return value

    def vega(self):
        ''' Return Vega of option. '''
        self.update_ttm()
        d1 = self.d1()
        vega = self.S0 * stats.norm.pdf(d1, 0.0, 1.0) * sqrt(self.T)
        return vega

    def imp_vol(self, C0, sigma_est=0.2):
        ''' Return implied volatility given option price. '''
        option = call_option(self.S0, self.K, self.t, self.M,
                             self.r, sigma_est)
        option.update_ttm()

        def difference(sigma):
            option.sigma = sigma
            return option.value() - C0
        iv = fsolve(difference, sigma_est)[0]
        return iv

In [7]:
#
#
# Black-Scholes-Merton Implied Volatilities of
# Call Options on the EURO STOXX 50
# Option Quotes from 30. September 2014
# Source: www.eurexchange.com, www.stoxx.com
# 03_stf/ES50_imp_vol.py
#
# (c) Dr. Yves J. Hilpisch
# Derivatives Analytics with Python
#
import numpy as np
import pandas as pd
#from BSM_imp_vol import call_option
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rcParams['font.family'] = 'serif'

# Pricing Data
pdate = pd.Timestamp('30-09-2014')

#
# EURO STOXX 50 index data
#

# URL of data file
es_url = 'http://www.stoxx.com/download/historical_values/hbrbcpe.txt'
# column names to be used
cols = ['Date', 'SX5P', 'SX5E', 'SXXP', 'SXXE',
        'SXXF', 'SXXA', 'DK5F', 'DKXF', 'DEL']
# reading the data with pandas
es = pd.read_csv(es_url,  # filename
                 header=None,  # ignore column names
                 index_col=0,  # index column (dates)
                 parse_dates=True,  # parse these dates
                 dayfirst=True,  # format of dates
                 skiprows=4,  # ignore these rows
                 sep=';',  # data separator
                 names=cols)  # use these column names
# deleting the helper column
del es['DEL']
S0 = es['SX5E']['30-09-2014']
r = -0.05

#
# Option Data
#
data = pd.HDFStore('./es50_option_data.h5', 'r')['data']

#
# BSM Implied Volatilities
#


def calculate_imp_vols(data):
    ''' Calculate all implied volatilities for the European call options
    given the tolerance level for moneyness of the option.'''
    data['Imp_Vol'] = 0.0
    tol = 0.30  # tolerance for moneyness
    for row in data.index:
        t = data['Date'][row]
        T = data['Maturity'][row]
        ttm = (T - t).days / 365.
        forward = np.exp(r * ttm) * S0
        if (abs(data['Strike'][row] - forward) / forward) < tol:
            call = call_option(S0, data['Strike'][row], t, T, r, 0.2)
            data.loc[row, 'Imp_Vol'] = call.imp_vol(data.loc[row, 'Call'])
    return data


#
# Graphical Output
#
markers = ['.', 'o', '^', 'v', 'x', 'D', 'd', '>', '<']


def plot_imp_vols(data):
    ''' Plot the implied volatilites. '''
    maturities = sorted(set(data['Maturity']))
    plt.figure(figsize=(10, 5))
    for i, mat in enumerate(maturities):
        dat = data[(data['Maturity'] == mat) & (data['Imp_Vol'] > 0)]
        plt.plot(dat['Strike'].values, dat['Imp_Vol'].values,
                 'b%s' % markers[i], label=str(mat)[:10])
    plt.grid()
    plt.legend()
    plt.xlabel('strike')
    plt.ylabel('implied volatility')

/opt/anaconda3/envs/trainingAI/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3172: UserWarning: Parsing '30-09-2014' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/opt/anaconda3/envs/trainingAI/lib/python3.9/site-packages/pandas/core/series.py:959: UserWarning: Parsing '30-09-2014' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  return self._get_value(key)


In [14]:
#
# Analyzing Euribor Interest Rate Data
# Source: http://www.emmi-benchmarks.eu/euribor-org/euribor-rates.html
# 03_stf/EURIBOR_analysis.py
#
# (c) Dr. Yves J. Hilpisch
# Derivatives Analytics with Python
#
import pandas as pd
#from GBM_returns import *

# Read Data for Euribor from Excel file


def read_euribor_data():
    ''' Reads historical Euribor data from Excel file, calculates log returns, 
    realized variance and volatility.'''
    EBO = pd.read_excel('./EURIBOR_current.xlsx',
                        index_col=0)
    EBO['returns'] = np.log(EBO['1w'] / EBO['1w'].shift(1))
    EBO = EBO.dropna()
    return EBO


# Plot the Term Structure
markers = [',', '-.', '--', '-']


def plot_term_structure(data):
    ''' Plot the term structure of Euribor rates. '''
    plt.figure(figsize=(10, 5))
    for i, mat in enumerate(['1w', '1m', '6m', '12m']):
        plt.plot(data[mat].index, data[mat].values,
                 'b%s' % markers[i], label=mat)
    plt.grid()
    plt.legend()
    plt.xlabel('strike')
    plt.ylabel('implied volatility')
    plt.ylim(0.0, plt.ylim()[1])

In [16]:
#
# Black-Scholes-Merton Implied Volatilities of
# Call Options on the DAX
# Quotes from 29 Apr 2011
# Source: www.eurexchange.com
# DAX_Imp_Vol.py
#
# (c) Dr. Yves J. Hilpisch
# Derivatives Analytics with Python
#
from pylab import *
import mpl_toolkits.mplot3d.axes3d as p3


#
# Option Data
#
S0 = 7514.46  # initial index level 29 Apr 2011
T = array((21., 49., 140., 231., 322.)) / 365.  # call option maturities
r = [0.0124, 0.0132, 0.015, 0.019, 0.0214]  # approx. short rates

# May 2011
K = array((7000, 7050, 7100, 7150, 7200, 7250, 7300, 7350, 7400, 7450,
         7500, 7550, 7600, 7650, 7700, 7750, 7800, 7850, 7900, 7950, 8000))
C1 = array((530.8, 482.9, 435.2, 388.5, 342.5, 297.8, 254.8, 213.7, 175.4,
          140.2, 108.7, 81.6, 59, 41.2, 27.9, 18.5, 12.1, 7.9, 5.1, 3.4, 2.3))

# June 2011
C2 = array((568.9, 524.5, 481.1, 438.7, 397.3, 357.5, 318.9, 281.9, 247, 214,
          183.3, 155.1, 129.3, 106.3, 86.1, 68.8, 54.1, 42, 32.2, 24.5, 18.4))

# Sep 2011
C3 = array((697.1, 657.9, 619.5, 581.8, 544.9, 509.1, 474.2, 440, 407.2, 375.4,
          344.8, 315.3, 287.5, 260.6, 235.5, 211.5, 189, 168, 148.8, 130.7,
          114.2))

# Dec 2011
C4 = array((811.5, 774.4, 737.9, 702.1, 666.9, 632.3, 598.5, 565.6, 533.5,
          502.1, 471.6, 442.1, 413.2, 385.6, 359, 333.2, 308.4, 284.9, 262.4,
          240.9, 220.4))

# Mar 2012
C5 = array((921.3, 885.4, 849.8, 814.7, 780.1, 746.6, 713.4, 680.8, 648.9,
          617.7, 587.1, 557.4, 528.6, 500.2, 472.6, 446, 419.9, 395, 370.4,
          347.1, 324.6))

#
# BSM Implied Volatilities
#
imv1 = []
imv2 = []
imv3 = []
imv4 = []
imv5 = []
BSM_Call_ImpVol = call_option
for j in range(len(K)):
    imv1.append(BSM_Call_ImpVol(S0, K[j], T[0], r[0], C1[j], 0.2))
    imv2.append(BSM_Call_ImpVol(S0, K[j], T[1], r[1], C2[j], 0.2))
    imv3.append(BSM_Call_ImpVol(S0, K[j], T[2], r[2], C3[j], 0.2))
    imv4.append(BSM_Call_ImpVol(S0, K[j], T[3], r[3], C4[j], 0.2))
    imv5.append(BSM_Call_ImpVol(S0, K[j], T[4], r[4], C5[j], 0.2))
imv1 = array(imv1)
imv2 = array(imv2)
imv3 = array(imv3)
imv4 = array(imv4)
imv5 = array(imv5)
imv = array((imv1, imv2, imv3, imv4, imv5))
print(imv)
#
# Graphical Output
#
## 2d Output
figure()
plot(K, imv[0] * 100, 'ro')
plot(K, imv[1] * 100, 'gx')
plot(K, imv[2] * 100, 'bv')
plot(K, imv[3] * 100, 'yD')
plot(K, imv[4] * 100, 'mh')
grid(True)
xlabel('Strike')
ylabel('Implied Volatility')

## 3d Output
k, t = meshgrid(K, T)
fig = figure()
plot = p3.Axes3D(fig)
plot.plot_wireframe(k, t, imv)
plot.set_xlabel('K')
plot.set_ylabel('T')
plot.set_zlabel('Implied Volatility')

[[<__main__.call_option object at 0x7fdd039137f0>


TypeError: unsupported operand type(s) for *: 'call_option' and 'int'

<Figure size 432x288 with 0 Axes>